In [1]:
#Imports
from datetime import datetime, timedelta
import time
from collections import namedtuple
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json
import numpy as np
from pandas.api.types import is_string_dtype
import os
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,Normalizer,FunctionTransformer,QuantileTransformer,PowerTransformer
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
import warnings
from sklearn.svm import SVR
import statsmodels.api as sm
warnings.filterwarnings("ignore")

In [2]:
buildings=pd.read_csv('buildings.csv')
equip_type=pd.read_csv('equip_type.csv')
equipments=pd.read_csv('equipments.csv')
local_data_types=pd.read_csv('local_data_types.csv')
point_type=pd.read_csv('point_type.csv')
points=pd.read_csv('points.csv')
tags=pd.read_csv('tags.csv')

In [5]:
see=pd.read_csv('Energy_data_reduced.csv')

In [6]:
buildings.head()

,id,org_id,name,sq_ft,timezone,status,equip_count,point_count,info.org,info.floors,...,info.sunend,info.geoCity,info.geoState,info.m2fstart,info.satstart,info.sunstart,info.yearBuilt,info.geoCountry,info.weatherRef,info.customerType
0,441,5,88737,551143.0,America/New_York,LIVE,20,1293,NaN,NaN,...,NaN,Bronx,NY,NaN,NaN,NaN,NaN,US,NaN,Multifamily
1,140,5,83483,21869.0,America/New_York,LIVE,13,354,NaN,NaN,...,NaN,New York,NY,NaN,NaN,NaN,NaN,US,NaN,Commercial Office
2,191,5,94414,198057.0,America/New_York,LIVE,7,19,NaN,NaN,...,23:00,Victor,NY,07:30,07:30,07:30,NaN,US,NaN,Commercial Retail
3,231,5,89496,NaN,America/New_York,LIVE,4,51,NaN,NaN,...,NaN,New York,NY,NaN,NaN,NaN,NaN,US,NaN,NaN
4,248,5,116742,127000.0,America/New_York,LIVE,7,34,NaN,NaN,...,NaN,New York,NY,NaN,NaN,NaN,NaN,US,NaN,Multifamily


In [21]:
buildings.shape

(229, 22)

In [7]:
buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 229 non-null    int64  
 1   org_id             229 non-null    int64  
 2   name               229 non-null    int64  
 3   sq_ft              201 non-null    float64
 4   timezone           229 non-null    object 
 5   status             229 non-null    object 
 6   equip_count        229 non-null    int64  
 7   point_count        229 non-null    int64  
 8   info.org           8 non-null      object 
 9   info.floors        9 non-null      float64
 10  info.m2fend        35 non-null     object 
 11  info.satend        30 non-null     object 
 12  info.sunend        32 non-null     object 
 13  info.geoCity       226 non-null    object 
 14  info.geoState      226 non-null    object 
 15  info.m2fstart      35 non-null     object 
 16  info.satstart      30 non-

In [8]:
equip_type.head()

,id,tag_name,name_long,name_abbr,active,flow_order,critical_point_types,sub_types,tags
0,12,ahu,Air Handling Unit,AHU,True,2,"[79, 85, 103, 159, 160]","[{'id': 1, 'equipment_type_id': 12, 'tag_name'...","['hvac', 'ahu']"
1,19,boiler,Boiler,BLR,True,1,"[88, 131, 92, 167, 121]","[{'id': 4, 'equipment_type_id': 19, 'tag_name'...","['hvac', 'boiler']"
2,20,chilledWaterPlant,Chilled Water Plant,CHWS,True,1,"[89, 132, 93, 116, 108]",[],"['hvac', 'chilledWaterPlant']"
3,21,chiller,Chiller,CH,True,1,"[89, 132, 93, 167, 121]","[{'id': 7, 'equipment_type_id': 21, 'tag_name'...","['hvac', 'chiller']"
4,22,condenser,Condenser,COND,True,1,[121],[],"['hvac', 'condenser']"


In [9]:
equip_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    39 non-null     int64 
 1   tag_name              39 non-null     object
 2   name_long             39 non-null     object
 3   name_abbr             39 non-null     object
 4   active                39 non-null     bool  
 5   flow_order            39 non-null     int64 
 6   critical_point_types  39 non-null     object
 7   sub_types             39 non-null     object
 8   tags                  39 non-null     object
dtypes: bool(1), int64(2), object(6)
memory usage: 2.6+ KB


In [10]:
equipments.head()

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,equip_dis,parent_equip,child_equip,tags
0,28797,441,boiler-1,1,Boiler,19,BLR,boiler,NaN,NaN,NaN,1,NaN,[28803],NaN,"['boiler', 'hvac']"
1,28798,441,boiler-2,2,Boiler,19,BLR,boiler,NaN,NaN,NaN,1,NaN,[28803],NaN,"['boiler', 'hvac']"
2,28799,441,boiler-3,3,Boiler,19,BLR,boiler,NaN,NaN,NaN,1,NaN,[28803],NaN,"['boiler', 'hvac']"
3,28800,441,boiler-4,4,Boiler,19,BLR,boiler,NaN,NaN,NaN,1,NaN,[28803],NaN,"['boiler', 'hvac']"
4,28801,441,boiler-5,5,Boiler,19,BLR,boiler,NaN,NaN,NaN,1,NaN,[28803],NaN,"['boiler', 'hvac']"


In [22]:
equipments.shape

(6085, 16)

In [23]:
equipments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6085 entries, 0 to 6084
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  6085 non-null   int64  
 1   building_id         6085 non-null   int64  
 2   equip_id            6085 non-null   object 
 3   suffix              6025 non-null   object 
 4   equip_type_name     6085 non-null   object 
 5   equip_type_id       6085 non-null   int64  
 6   equip_type_abbr     6085 non-null   object 
 7   equip_type_tag      6085 non-null   object 
 8   equip_subtype_name  2283 non-null   object 
 9   equip_subtype_id    2283 non-null   float64
 10  equip_subtype_tag   2283 non-null   object 
 11  flow_order          6085 non-null   int64  
 12  equip_dis           3012 non-null   object 
 13  parent_equip        884 non-null    object 
 14  child_equip         442 non-null    object 
 15  tags                6021 non-null   object 
dtypes: flo

In [12]:
local_data_types.head()

,type,count
0,Virtual,7284
1,Zone Temperature,5186
2,Status,2569
3,Discharge Air Flow,2522
4,Command,1704


In [13]:
local_data_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    246 non-null    object
 1   count   246 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


In [14]:
point_type.head()

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
0,124,Occupied Heating Setpoint,True,1,"[15, 2, 4, 62, 51, 55]","['air', 'sp', 'temp', 'zone', 'heating', 'occ']",NaN
1,118,Outside Air Carbon Dioxide,True,6,"[15, 8, 1, 24]","['air', 'co2', 'sensor', 'outside']",NaN
2,130,Return Air Temperature Setpoint,True,1,"[15, 2, 4, 63]","['air', 'sp', 'temp', 'return']",NaN
3,84,Dual-Temp Coil Discharge Air Temperature,True,1,"[15, 36, 47, 1, 4, 250]","['air', 'discharge', 'dualTemp', 'sensor', 'te...",NaN
4,81,Reheat Coil Discharge Air Temperature,True,1,"[15, 36, 44, 1, 4, 250]","['air', 'discharge', 'reheats', 'sensor', 'tem...",NaN


In [15]:
point_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               647 non-null    int64  
 1   tag_name         647 non-null    object 
 2   active           647 non-null    bool   
 3   measurement_id   647 non-null    int64  
 4   tag_set_ids      647 non-null    object 
 5   tags             647 non-null    object 
 6   default_unit_id  0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(3)
memory usage: 31.1+ KB


In [16]:
points.head()

,id,building_id,last_updated,first_updated,name,description,units,tagged_units,raw_unit_id,value,...,point_type_id,measurement_id,datasource_hash,topic,equip_id,state_text.1,state_text.2,state_text.3,state_text.4,state_text.5
0,310029,441,1.560571e+12,1.550542e+12,245 E 149th St. Boiler-1 Fault Code,NaN,NaN,NaN,23,0,...,434,18,b115b5053493748f7076ad09f69c4456,nyserda+0000088737+boiler-1+245 E 149th St. Bo...,28797,NaN,NaN,NaN,NaN,NaN
1,310009,441,1.560485e+12,1.550701e+12,225 E 149th St. Boiler-1 Exhaust Temp,NaN,F,f,2,183.55,...,86,1,4a1a11502acee3078352da312c40c262,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,NaN,NaN,NaN,NaN,NaN
2,310010,441,1.560571e+12,1.550504e+12,225 E 149th St. Boiler-1 Fault Code,NaN,NaN,NaN,23,0,...,434,18,55882633fe8fb5aac7dac643c99b1af5,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,NaN,NaN,NaN,NaN,NaN
3,310011,441,1.560485e+12,1.550504e+12,225 E 149th St. Boiler-1 FFWD Temp,NaN,NaN,NaN,23,46.90,...,434,18,831a8df09b848f53e807fa0204c45eaa,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,NaN,NaN,NaN,NaN,NaN
4,310012,441,1.560485e+12,1.550504e+12,225 E 149th St. Boiler-1 Fire Rate In,NaN,NaN,NaN,23,91,...,434,18,aa147c829c7ed55b6e93cdbbb8b43d55,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,NaN,NaN,NaN,NaN,NaN


In [9]:
see.loc[see['address_1'] == '245 E 149th St.']

,Unnamed: 0.1,Unnamed: 0,property_id,property_name,parent_property_id,parent_property_name,year_ending,address_1,city,postcode,...,alert_data_center_issue_with,alert_gross_floor_area_is,alert_energy_meter_has_less,alert_energy_meter_has,alert_energy_no_meters,alert_energy_meter_has_single,property_gfa_self_reported,property_gfa_calculated,property_gfa_calculated_1,data_quality_checker_run


In [6]:
see.head()

,Unnamed: 0.1,Unnamed: 0,property_id,property_name,parent_property_id,parent_property_name,year_ending,address_1,city,postcode,...,alert_data_center_issue_with,alert_gross_floor_area_is,alert_energy_meter_has_less,alert_energy_meter_has,alert_energy_no_meters,alert_energy_meter_has_single,property_gfa_self_reported,property_gfa_calculated,property_gfa_calculated_1,data_quality_checker_run
0,0,0,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2020-12-31T00:00:00.000,1155 Avenue of the Americas,Manhattan,10036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,8139,200,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2020-12-31T00:00:00.000,200 East 42nd St.,Manhattan,10017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,8604,114,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2020-12-31T00:00:00.000,114 West 47th st,Manhattan,10036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,8841,733,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2020-12-31T00:00:00.000,733 Third Avenue,Manhattan,10017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,11809,Conde Nast Building,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2020-12-31T00:00:00.000,4 Times Square,Manhattan,10036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44390 entries, 0 to 44389
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               44390 non-null  int64  
 1   building_id      44390 non-null  int64  
 2   last_updated     44342 non-null  float64
 3   first_updated    44342 non-null  float64
 4   name             14658 non-null  object 
 5   description      31843 non-null  object 
 6   units            30568 non-null  object 
 7   tagged_units     12606 non-null  object 
 8   raw_unit_id      44390 non-null  int64  
 9   value            44274 non-null  object 
 10  type             44390 non-null  object 
 11  point_type_id    44390 non-null  int64  
 12  measurement_id   44390 non-null  int64  
 13  datasource_hash  44390 non-null  object 
 14  topic            44390 non-null  object 
 15  equip_id         44390 non-null  int64  
 16  state_text.1     1814 non-null   object 
 17  state_text.2

In [18]:
tags.head()

,id,name,definition,def_source,def_url,category
0,120,battery,A container that stores chemical energy that c...,brick,https://brickschema.org/ontology/1.1/classes/B...,NaN
1,191,exhaustVAV,A device that regulates the volume of air bein...,onboard,NaN,NaN
2,193,oil,"A viscous liquid derived from petroleum, espec...",brick,https://brickschema.org/ontology/1.2/classes/Oil/,NaN
3,114,fumeHood,A fume-collection device mounted over a work s...,brick,https://brickschema.org/ontology/1.1/classes/F...,NaN
4,118,limit,A parameter that places a lower or upper bound...,brick,https://brickschema.org/ontology/1.1/classes/L...,Point Class


In [19]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          243 non-null    int64 
 1   name        243 non-null    object
 2   definition  243 non-null    object
 3   def_source  243 non-null    object
 4   def_url     139 non-null    object
 5   category    33 non-null     object
dtypes: int64(1), object(5)
memory usage: 11.5+ KB
